In [ ]:
import numpy as np
import tifffile
from patchify import patchify
import os
from PIL import Image
from osgeo import gdal

# Define the input and output directories
input_dir = 'input_directory'
output_dir = 'output_directory'

# Define the patch size
patch_size = (1280, 1280)

# Loop over the GeoTIFF images in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):
        # Load the GeoTIFF image using tifffile
        image = tifffile.imread(os.path.join(input_dir, filename))

        # Pad the input image to make its size divisible by the patch size
        padded_size = ((image.shape[0] + patch_size[0] - 1) // patch_size[0]) * patch_size[0], \
                      ((image.shape[1] + patch_size[1] - 1) // patch_size[1]) * patch_size[1]
        padding = ((0, padded_size[0] - image.shape[0]), (0, padded_size[1] - image.shape[1]), (0, 0))
        padded_image = np.pad(image, padding, mode='constant')

        # Extract patches using patchify
        patches = patchify(padded_image, (1280, 1280, 3), 1280)

        # Get the geotransform of the input image
        ds = gdal.Open(os.path.join(input_dir, filename))
        geotransform = ds.GetGeoTransform()

        # Create the output subdirectory for the image
        output_subdir = os.path.join(output_dir, os.path.splitext(filename)[0])
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        # Loop over the patches and save them as JPEG/PNG images using Pillow
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                patch = np.squeeze(patches[i, j])
                # Get the upper-left corner of the patch in geographic coordinates
                x = geotransform[0] + j * 1280 * geotransform[1] + i * 1280 * geotransform[2]
                y = geotransform[3] + j * 1280 * geotransform[4] + i * 1280 * geotransform[5]
                # Get the lower-right corner of the patch in geographic coordinates
                a = x + 1280 * geotransform[1]
                b = y + 1280 * geotransform[5]
                # Define the path and filename for the output file
                filename = f"{x}_{y}_{a}_{b}.jpg"
                path = os.path.join(output_subdir, filename)
                # Convert the patch to a PIL Image object and save as JPEG/PNG
                img = Image.fromarray(patch, mode='RGB')
                img.save(path, format='JPEG')

